# LAB 3: NUMBA 
### SGD Optimisation with Numba
### AI and Machine Learning // Suchkova Natalia М8О-114М-22
09.11.2022 @ MAI IT-Center

In [250]:
import numpy as np
from numba import njit

from typing import Tuple, Mapping

import matplotlib.pyplot as plt
from IPython import display

In [251]:
class Himmelblau:
    
    def function(x: np.ndarray) -> np.float64:
        return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
    
    def gradient(x: np.ndarray) -> np.array:
        return np.array([4 * x[0] * (x[0]**2 + x[1] - 11) \
                         + 2 * (x[0] + x[1]**2 - 7),
                         2 * (x[0]**2 + x[1] - 11) \
                         + 4 * x[1] * (x[0] + x[1]**2 - 7)])

class Rosenbrock: 
    
    def function(x: np.ndarray, b: int = 100) -> np.float64:
        return b * (x[1] - x[0]**2)**2 + (x[0] - 1)**2

    def gradient(x: np.ndarray, b: int = 100) -> np.array:

        return np.array([2 * (x[0] - 1) \
                         - 4 * b * x[0] * (x[1] - x[0]**2),
                         2 * b * (x[1] - x[0]**2)])
    
class Rastrigin:
    
    def function(x: np.ndarray, A: int = 10) -> np.float64:
        return list(x**2 - A * np.cos(2 * np.pi * x))[0] # + A
    
    def gradient(x: np.float32, A: int = 10) -> np.float64:
            return 2 * x + A * 2 * np.pi * np.sin(2 * np.pi * x)

Упростим еще немного код, который был.

In [43]:
# def classic_GD (
#                 function: Mapping, gradient_of_function: Mapping,
#                 start: np.ndarray, learning_rate: float = 0.01, 
#                 n_iter: int = 100, tolerance: float = 1e-5
#                 ) -> Tuple [np.ndarray, np.float32]:
    
#     """ 
#     Args:
#         function (Mapping): минимизруемая функция
#         gradient_of_function (Mapping): градиент минимизируемой функции
#         start (np.ndarray): рандомная стартовая точка
#         learning_rate (float): шаг минимизации
#         n_iter (int): количество итераций градиентного спуска
#         tolerance (float): минимальное допустимое изменение 
#                     значения минимизируемой величины
#     Return:
#         tuple with found minimum coordinates, found minimum function value, 
#         and plotting data list with multiple np.ndarray aka dots for plotting
    
#    """ 
        
#     current_point = start.copy()
#     current_point = current_point.astype('float64')

#     for iter in range(n_iter):
#         diff = learning_rate * -gradient_of_function(current_point)

#         if np.all(np.abs(diff) <= tolerance):
#             print(f'\033[1mEarly stopping!!\033[0m')
#             break

#         iter_count = iter + 1    
#         current_point += diff

#     print(f' Finished in {iter_count} iterations\n', \
#           f'Minimum point coordinates \033[1m{current_point}\033[0m,', \
#           f'with function value = \033[1m{round(function(current_point), 4)}\033[0m')


#     return current_point, function(current_point)

In [190]:
def classic_GD (
                function: Mapping, gradient_of_function: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100,
                ) -> Tuple [np.ndarray, np.float32]:
    
    """ 
    Args:
        function (Mapping): минимизруемая функция
        gradient_of_function (Mapping): градиент минимизируемой функции
        start (np.ndarray): рандомная стартовая точка
        learning_rate (float): шаг минимизации
        n_iter (int): количество итераций градиентного спуска

    Return:
        tuple with found minimum coordinates, found minimum function value, 
        and plotting data list with multiple np.ndarray aka dots for plotting
    
   """    
    current_point = start.copy()
    current_point = current_point.astype('float64')

    for iter in range(n_iter):   
        current_point = current_point - learning_rate * gradient_of_function(current_point)

    return current_point, function(current_point)

In [191]:
%%time
GD_Him = classic_GD(Himmelblau.function, Himmelblau.gradient, np.array([-100, -100]), learning_rate=0.00001, n_iter=7000)

Wall time: 89.7 ms


In [194]:
print(f'Minimum point coordinates \033[1m{GD_Him[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Him[1], 4)}\033[0m')

Minimum point coordinates [-3.78698073 -3.29532809], with function value = 0.0073


In [195]:
%%time
GD_Rast = classic_GD(Rastrigin.function, Rastrigin.gradient, np.array([100, 100]), learning_rate=0.1, n_iter=9000)

Wall time: 102 ms


In [196]:
print(f'Minimum point coordinates \033[1m{GD_Rast[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Rast[1], 4)}\033[0m')

Minimum point coordinates [0.76548588 0.76548588], with function value = -0.3855


In [221]:
%%time
GD_Ros = classic_GD(Rosenbrock.function, Rosenbrock.gradient, np.array([10, 10]), 
           tolerance = 1e-7, learning_rate=0.00001, n_iter=200000)

Wall time: 2.56 s


In [222]:
print(f'Minimum point coordinates \033[1m{GD_Ros[0]}\033[0m,', \
      f'with function value = \033[1m{round(GD_Ros[1], 4)}\033[0m')

Minimum point coordinates [3.02969959 9.18234049], with function value = 4.1207


In [252]:
def GD_AdaGrad(
            function: Mapping, gradient: Mapping, start: np.ndarray,
            gamma: np.float64 = 0.1, n_iter: int = 100, 
            learning_rate: float = 0.01, tolerance=1e-06, 
            dtype="float64", rand_state: int = 12
            ) -> Tuple [np.ndarray, np.ndarray]:
    """ 
        function (Mapping):  минимизруемая функция
        gradient (Mapping): градиент заданной выше функции
        start (np.ndarray): рандомная стартовая точка/точки
        gamma (float): скорость затухания скользащих средних ф-ии потерь
        n_iter (int): количество итераций градиентного спуска
        learning_rate (float): шаг минимизации
        tolerance (float): минимальное допустимое изменение 
                           значения минимизируемой величины
        dtype (str): тип данных
        rand_state (int): зерно рандомайзера
        
    """
    dtype_ = np.dtype(dtype)

    cur_point = start.copy()
    cur_point = cur_point.astype('float64')

    G = np.zeros(cur_point.shape, dtype=dtype_) 
   
    for iter in range(n_iter):
        loss = gradient(cur_point)
        G = gamma * G + loss ** 2

        if np.all(np.abs(loss/np.sqrt(G)) <= tolerance):
            print(f'\033[1mEarly stopping!!\033[0m')
            break

        cur_point -= (loss/np.sqrt(G)) * learning_rate
        iter_count = iter + 1
        
    print(f' Finished in {iter_count} iterations\n')

    return cur_point, function(cur_point)

In [257]:
%%time
Him_Ada = GD_AdaGrad(Himmelblau.function, Himmelblau.gradient, np.array([100, 100]),
                     gamma=0.8, learning_rate=0.01, n_iter=30000)

 Finished in 30000 iterations

Wall time: 1.72 s


In [258]:
print(f'Minimum point coordinates \033[1m{Him_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Him_Ada[1], 6)}\033[0m')

Minimum point coordinates [3.00223411 2.00223281], with function value = 0.000369


In [259]:
%%time
Rast_Ada = GD_AdaGrad(Rastrigin.function, Rastrigin.gradient, np.array([-10, 100]), 
                      tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)

 Finished in 2000000 iterations

Wall time: 1min 36s


In [ ]:
print(f'Minimum point coordinates \033[1m{Rast_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Rast_Ada[1], 6)}\033[0m')

In [56]:
%%time
Ros_Ada = GD_AdaGrad(Rosenbrock.function, Rosenbrock.gradient, np.array([-10, -10]),
                     gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)

 Finished in 30000 iterations

Minimum point coordinates [1.00008264 0.9954266 ], with function value = 0.002246
Wall time: 1.44 s


In [ ]:
print(f'Minimum point coordinates \033[1m{Ros_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Ros_Ada[1], 6)}\033[0m')

### With Numba

In [223]:
@njit(fastmath=True)
def Himmelblau(x: np.ndarray) -> np.float64:
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2
@njit(fastmath=True)
def Him_grad(x: np.ndarray) -> np.array:
    return np.array([4 * x[0] * (x[0]**2 + x[1] - 11) \
                     + 2 * (x[0] + x[1]**2 - 7),
                     2 * (x[0]**2 + x[1] - 11) \
                         + 4 * x[1] * (x[0] + x[1]**2 - 7)])
@njit(fastmath=True)
def Rosenbrock(x: np.ndarray, b: int = 100) -> np.float64:
    return b * (x[1] - x[0]**2)**2 + (x[0] - 1)**2
@njit(fastmath=True)
def Rosen_grad(x: np.ndarray, b: int = 100) -> np.array:
    return np.array([2 * (x[0] - 1) \
                     - 4 * b * x[0] * (x[1] - x[0]**2),
                         2 * b * (x[1] - x[0]**2)])
@njit(fastmath=True)
def Rastrigin(x: np.ndarray, A: int = 10) -> np.float64:
    return list(x**2 - A * np.cos(2 * np.pi * x))[0] # + A
@njit(fastmath=True)
def Rast_grad(x: np.float32, A: int = 10) -> np.float64:
        return 2 * x + A * 2 * np.pi * np.sin(2 * np.pi * x)

In [224]:
@njit
def classic_GD (
                function: Mapping, gradient_of_function: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100
                ) -> Tuple [np.ndarray, np.float32]:
    
    """ 
    Args:
        function (Mapping): минимизруемая функция
        gradient_of_function (Mapping): градиент минимизируемой функции
        start (np.ndarray): рандомная стартовая точка
        learning_rate (float): шаг минимизации
        n_iter (int): количество итераций градиентного спуска
    Return:
        tuple with found minimum coordinates, found minimum function value, 
        and plotting data list with multiple np.ndarray aka dots for plotting
    
   """   
    current_point = start.copy()
    current_point = current_point.astype('float64')
    
    for iter in range(n_iter):   
        current_point = current_point - learning_rate * gradient_of_function(current_point)

    return current_point, function(current_point)

In [235]:
%%time
GD_Him_GPU = classic_GD(Himmelblau, Him_grad, np.array([-100, -100]), learning_rate=0.00001, n_iter=9500)

Wall time: 2.99 ms


In [236]:
print(f'Minimum point coordinates \033[1m{GD_Him_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Him_GPU[1], 4)}\033[0m')

Minimum point coordinates [-3.78059997 -3.28525589],
with function value = 0.0002


In [243]:
%%time
GD_Rast_GPU = classic_GD(Rastrigin, Rast_grad, np.array([10, 10]), learning_rate=0.1, n_iter=200)

Wall time: 0 ns


In [244]:
print(f'Minimum point coordinates \033[1m{GD_Rast_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Rast_GPU[1], 4)}\033[0m')

Minimum point coordinates [-1.78314385 -1.78314385],
with function value = 1.1121


In [248]:
%%time
GD_Ros_GPU = classic_GD(Rosenbrock, Rosen_grad, np.array([10, 10]), learning_rate=0.00001, n_iter=200000)

Wall time: 113 ms


In [249]:
print(f'Minimum point coordinates \033[1m{GD_Ros_GPU[0]}\033[0m,')
print(f'with function value = \033[1m{round(GD_Ros_GPU[1], 4)}\033[0m')

Minimum point coordinates [3.02969959 9.18234049],
with function value = 4.1207


In [95]:
@njit
def GD_AdaGrad (
                function: Mapping, gradient: Mapping,
                start: np.ndarray, learning_rate: float = 0.01, 
                n_iter: int = 100, tolerance: float = 1e-5,
                gamma: np.float64 = 0.1, dtype="float64", 
                rand_state: int = 12
                ) -> Tuple [np.ndarray, np.float64]:
    
    """ 
    Args:
        function (Mapping):  минимизруемая функция
        gradient (Mapping): градиент заданной выше функции
        start (np.ndarray): рандомная стартовая точка/точки
        gamma (float): скорость затухания скользащих средних ф-ии потерь
        n_iter (int): количество итераций градиентного спуска
        learning_rate (float): шаг минимизации
        tolerance (float): минимальное допустимое изменение 
                           значения минимизируемой величины
        dtype (str): тип данных
        rand_state (int): зерно рандомайзера
    Return: 
        tuple with found minimum point coordinates, 
        funcation value at this point
        
   """  
    cur_point = start.copy()
    cur_point = cur_point.astype('float64')
    
    G = np.zeros(cur_point.shape, dtype=np.float64)
    
    for iter in range(n_iter):
        loss = gradient(cur_point)
        G = gamma * G + loss ** 2

        if np.all(np.abs(loss/np.sqrt(G)) <= tolerance):
            print(f'\033[1mEarly stopping!!\033[0m')
            break

        cur_point -= (loss/np.sqrt(G)) * learning_rate
        iter_count = iter + 1

    print(f' Finished in {iter_count} iterations\n')

    return cur_point, function(cur_point)

In [96]:
%%time
Him_Ada = GD_AdaGrad(Himmelblau, Him_grad, np.array([100, 100]),
                     gamma=0.8, learning_rate=0.01, n_iter=30000)

print(f'Minimum point coordinates \033[1m{Him_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Him_Ada[1], 6)}\033[0m')

 Finished in 30000 iterations

Minimum point coordinates [3.00223411 2.00223281], with function value = 0.000369
Wall time: 3.52 s


In [97]:
%%time
Rast_Ada = GD_AdaGrad(Rastrigin, Rast_grad, np.array([-10, 100]), 
                      tolerance=1e-4, gamma=0.2, learning_rate=0.51, n_iter=2000000)
print(f'Minimum point coordinates \033[1m{Rast_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Rast_Ada[1], 6)}\033[0m')

 Finished in 2000000 iterations

Minimum point coordinates [-4.7446343  19.65106439], with function value = 22.848628
Wall time: 5.8 s


In [98]:
%%time
Ros_Ada = GD_AdaGrad(Rosenbrock, Rosen_grad, np.array([-10, -10]),
                     gamma=0.9, tolerance = 1e-3, learning_rate=0.01, n_iter=30000)
print(f'Minimum point coordinates \033[1m{Ros_Ada[0]}\033[0m,', 
      f'with function value = \033[1m{round(Ros_Ada[1], 6)}\033[0m')

 Finished in 30000 iterations

Minimum point coordinates [1.00008264 0.9954266 ], with function value = 0.002246
Wall time: 2.6 s
